# Ranking

Second Assignement of Learning with Massive Data. 


# Usefull Function

In [15]:
import pandas as pd

def score_table(score):
    df = pd.DataFrame(score)

    df.index = ["Query " + str(i+1) for i in range(len(df.index))]
    df.columns = ["Document " + str(i+1) for i in range(len(df.columns))]

    return df


In [1]:
import json
import numpy as np
import string

path_name_documents = './Databases/prova/gigi.jsonl'
path_name_query = './Databases/prova/query.jsonl'


def readFile(path_name):
    # Load the JSONL file into a list
    with open(path_name, 'r') as f:
        lines = f.readlines()

    # Convert each JSON object into a dictionary
    dicts = [json.loads(line) for line in lines]

    # Convert the dictionaries into arrays and stack them vertically
    arrays = np.vstack([np.array(list(d.values())) for d in dicts])

    # Convert the arrays into a list of lists
    text = arrays.tolist()
    
    return text

documents = readFile(path_name_documents)
#print(documents)
queries = readFile(path_name_query)
#print(query)


# Tokenize

In [2]:
import json
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer


stop_words = set(stopwords.words('english'))

def stemmingLemming(filtered_tokens):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    # Perform stemming or lemmatization on filtered tokens
    
    filtered_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    filtered_tokens = [stemmer.stem(token) for token in filtered_tokens]

    return filtered_tokens
    
 
    

def tokenize(path_name):
    
    with open(path_name, "r") as f:
        data = f.readlines()

        # Create an empty list to store the tokenized documents
        tokenized_docs = []

        # Loop through each line in the JSONL file
        for line in data:
            # Parse the JSON string into a Python dictionary
            doc = json.loads(line)

            # Extract the text from the dictionary
            text = doc['text']
            text = text.lower()  # Convert to lowercase
            #text = re.sub(r'\d+', '', text)  # Remove all numbers
            text = text.translate(str.maketrans('', '', string.punctuation))  # Remove all punctuation

            # Tokenize the text using NLTK
            tokens = word_tokenize(text)
            tokensStemLem = stemmingLemming(tokens)

            # Add the tokenized document to the list
            tokenized_docs.append(tokensStemLem)

        # Print the tokenized documents
    return tokenized_docs

tokenized_docs = tokenize(path_name_documents)


tokenized_query = tokenize(path_name_query)




# Sparse Vectors



# TF-IDF 

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

def calculateTFIDF(tokenized_docs):
    # Initialize the TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the tokenized documents into a TF-IDF matrix
    tfidf_matrix = vectorizer.fit_transform([' '.join(doc) for doc in tokenized_docs])

    # Get the feature names (tokens)
    feature_names = vectorizer.get_feature_names()

    # Return the TF-IDF matrix and the feature names
    return tfidf_matrix, feature_names,vectorizer
 
    
    
def calculateTFIDFQuery(tokenized_query,vectorizer):
    # Initialize the TfidfVectorizer
    #vectorizer = TfidfVectorizer()

    # Fit and transform the tokenized documents into a TF-IDF matrix
    tfidf_matrix = vectorizer.transform([' '.join(doc) for doc in tokenized_query])

    # Get the feature names (tokens)
    feature_names = vectorizer.get_feature_names()

    # Return the TF-IDF matrix and the feature names
    return tfidf_matrix, feature_names

    

tfidf_matrix_docs, feature_names_docs,vectorizer  = calculateTFIDF(tokenized_docs)
tfidf_matrix_query, feature_names_query  = calculateTFIDFQuery( tokenized_query,vectorizer)



# Dot Product for Sparse Vectors

In [16]:
import numpy as np


def calculate_Dot_Product_Sparse(tfidf_matrix_docs, tfidf_matrix_query):
    matrix_docs = tfidf_matrix_docs.toarray()
    matrix_query = tfidf_matrix_query.toarray()
    num_queries = matrix_query.shape[0]
    num_docs = matrix_docs.shape[0]
    sparse_score_results = []
    
    for query in range(num_queries):
        sparse_score_docs = []
        for doc in range(num_docs):
            dot_result = np.dot(matrix_query[query], matrix_docs[doc])
            sparse_score_docs.append(dot_result)
        sparse_score_results.append(sparse_score_docs)
        
    return sparse_score_results



sparse_score_results = calculate_Dot_Product_Sparse(tfidf_matrix_docs,tfidf_matrix_query)
sparse_score_table = score_table(sparse_score_results)
print(sparse_score_table)
    


         Document 1  Document 2  Document 3  Document 4  Document 5  \
Query 1    0.183441    0.339340    0.135609    0.366428    0.377008   
Query 2    0.219293    0.149213    0.102671    0.199109    0.225653   
Query 3    0.037007    0.015821    0.090983    0.018307    0.094116   
Query 4    0.021409    0.000000    0.026377    0.000000    0.016987   
Query 5    0.079071    0.015343    0.010202    0.053696    0.049601   
Query 6    0.072174    0.062264    0.034060    0.096203    0.083014   
Query 7    0.074665    0.016662    0.030792    0.115323    0.130225   
Query 8    0.202117    0.220873    0.101320    0.287763    0.256384   

         Document 6  Document 7  Document 8  Document 9  Document 10  \
Query 1    0.215299    0.168381    0.218235    0.097616     0.334944   
Query 2    0.079196    0.171523    0.147755    0.229581     0.157818   
Query 3    0.057963    0.094272    0.006934    0.027960     0.016734   
Query 4    0.000000    0.089043    0.052155    0.000000     0.000000   


# Dense Vector

In [17]:

from sentence_transformers import SentenceTransformer

def create_dense_vector(document):
    # Load the all-MiniLM-L6-v2 model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    
    # Create a dense vector representation of the document
    dense_vector = model.encode(document, convert_to_tensor=True)
    
    
    return dense_vector


dense_vector_document = create_dense_vector(documents)
dense_vector_query = create_dense_vector(queries)




# Dot Product for Dense Vector

In [18]:
def calculate_Dot_Product_Dense(dense_vector_document, dense_vector_query):
    num_queries = dense_vector_query.shape[0]
    num_docs = dense_vector_document.shape[0]
    dense_score_results = []
    
    for query in range(num_queries):
        dense_score_docs = []
        for doc in range(num_docs):
            dot_result = np.dot(dense_vector_query[query], dense_vector_document[doc])
            dense_score_docs.append(dot_result)
        dense_score_results.append(dense_score_docs)
        
    return dense_score_results


dense_score_results = calculate_Dot_Product_Dense(dense_vector_document, dense_vector_query)
dense_score_table = score_table(dense_score_results)
print(dense_score_table)
    

         Document 1  Document 2  Document 3  Document 4  Document 5  \
Query 1    0.077958    0.038245    0.133962    0.106983    0.226969   
Query 2    0.099590    0.070468    0.066888    0.096966    0.279706   
Query 3    0.115690    0.103854    0.194563    0.120762    0.322730   
Query 4    0.144724    0.175452    0.131543    0.226517    0.249557   
Query 5    0.229337    0.190014    0.095667    0.222562    0.330017   
Query 6    0.092435    0.147272    0.173545    0.096595    0.171428   
Query 7    0.174007    0.177980    0.213392    0.104680    0.231447   
Query 8    0.129492    0.135007    0.152630    0.118768    0.170204   

         Document 6  Document 7  Document 8  Document 9  Document 10  \
Query 1    0.126819    0.324864    0.217792    0.038656     0.038245   
Query 2    0.121130    0.310443    0.200889    0.052011     0.070468   
Query 3    0.129222    0.261768    0.200194    0.167548     0.103854   
Query 4    0.214995    0.339539    0.206882    0.168591     0.175452   


# Calculate the final score

In [10]:
def sum_vectors_score(dense_score, sparse_score):
    final_score = []
    for i in range(len(dense_score)):
        row = []
        for j in range(len(dense_score[i])):
            row.append(dense_score[i][j] + sparse_score[i][j])
        final_score.append(row)
    return final_score

final_score = sum_vectors_score(dense_score_results,sparse_score_results)

    
def table_final_score(final_score):
    df = pd.DataFrame(final_score)

    df.index = ["Query " + str(i+1) for i in range(len(df.index))]
    df.columns = ["Document " + str(i+1) for i in range(len(df.columns))]

    return df

df = table_final_score(final_score)
print(df)
    

         Document 1  Document 2  Document 3  Document 4  Document 5  \
Query 1    0.261399    0.377585    0.269571    0.473410    0.603978   
Query 2    0.318883    0.219681    0.169559    0.296075    0.505359   
Query 3    0.152697    0.119676    0.285546    0.139068    0.416846   
Query 4    0.166132    0.175452    0.157920    0.226517    0.266544   
Query 5    0.308408    0.205356    0.105869    0.276258    0.379617   
Query 6    0.164609    0.209536    0.207605    0.192798    0.254442   
Query 7    0.248673    0.194642    0.244185    0.220003    0.361672   
Query 8    0.331609    0.355880    0.253950    0.406531    0.426588   

         Document 6  Document 7  Document 8  Document 9  Document 10  \
Query 1    0.342117    0.493245    0.436028    0.136272     0.373188   
Query 2    0.200326    0.481966    0.348644    0.281591     0.228287   
Query 3    0.187185    0.356040    0.207127    0.195508     0.120588   
Query 4    0.214995    0.428582    0.259037    0.168591     0.175452   


# Tok-k documents 

In [20]:
def found_top_k(final_score, k):
    top_k = []
    for scores in final_score:
        sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)[:k]
        top_k.append([(index, score) for index, score in sorted_scores])
    return top_k


list_top_k = found_top_k(final_score,3) 

results_k = table_final_score(list_top_k)
print(results_k)

                       Document 1                Document 2  \
Query 1   (4, 0.6039777237570092)   (6, 0.4932448673431095)   
Query 2   (4, 0.5053588847080347)  (6, 0.48196612801000327)   
Query 3   (4, 0.4168460690526282)  (6, 0.35603950481593094)   
Query 4  (6, 0.42858238921732694)  (4, 0.26654419827789694)   
Query 5   (4, 0.3796173551092559)   (5, 0.3276194630114711)   
Query 6  (8, 0.26717389429683513)  (6, 0.25652085826633153)   
Query 7   (6, 0.4446225101574543)  (8, 0.37113577447895024)   
Query 8   (4, 0.4265884576560837)   (8, 0.4092677309067976)   

                       Document 3  
Query 1   (3, 0.4734102674770713)  
Query 2   (7, 0.3486439400161163)  
Query 3  (2, 0.28554640233045037)  
Query 4  (7, 0.25903672514783505)  
Query 5   (0, 0.3084077697406177)  
Query 6   (4, 0.2544423221988228)  
Query 7   (4, 0.3616721029467196)  
Query 8  (3, 0.40653094177607463)  
